In [ ]:
## For Everyone and Anyone !! ** PLease Upvote if helped !! **

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np

## for plotting
import matplotlib.pyplot as plt
import seaborn as sns

## for text processing
from dateutil import parser
import string
import re
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk.stem.porter import PorterStemmer
from spacy.lang.en.stop_words import STOP_WORDS

from gensim.models import KeyedVectors

## for language detection
## warnings !!
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import spacy.cli/
# spacy.cli.download("en_core_web_lg")
import en_core_web_lg
nlp = en_core_web_lg.load()

from nltk.corpus import stopwords
eng_stopwords = set(stopwords.words("english"))

In [ ]:
# ! pip install langdetect
# ! pip install translate
# ! pip install gensim
# ! pip install googletrans

In [ ]:
train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

submission = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

print('Training shape : {}'.format(train.shape))
print('Testing shape : {}'.format(test.shape))

In [ ]:
train.head()

In [ ]:
## Feature Engineering :-------------->

## Feature Engg Prior to Cleaning :  to a generate some relevant features and then clenaing the irrelevnt data

In [ ]:
# Feature Extraction from the job discription :

print('* ------------------------- for train data-------------------------------- *')

# the relevance of these feature will be tested on the later stages !!

train['word_count'] = train["excerpt"].apply(lambda x: len(str(x).split(" ")))
train['char_count'] = train["excerpt"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))
train["num_unique_words"] = train["excerpt"].apply(lambda x: len(set(str(x).split())))
train["num_stopwords"] = train["excerpt"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
train["num_punctuations"] =train['excerpt'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
train["num_words_upper"] = train["excerpt"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
train["num_words_title"] = train["excerpt"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
train["mean_word_len"] = train["excerpt"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

print('* ------------------------- for test data-------------------------------- *')

test['word_count'] = test["excerpt"].apply(lambda x: len(str(x).split(" ")))
test['char_count'] = test["excerpt"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))
test["num_unique_words"] = test["excerpt"].apply(lambda x: len(set(str(x).split())))
test["num_stopwords"] = test["excerpt"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
test["num_punctuations"] =test['excerpt'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
test["num_words_upper"] = test["excerpt"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
test["num_words_title"] = test["excerpt"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
test["mean_word_len"] = test["excerpt"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [ ]:
train.head(2)

In [ ]:
## Data Cleaning :------------->

In [ ]:
def is_valid_date(date_str): # sub function
    try:
        parser.parse(date_str)
        return True
    except:
        return False
    
def date_removal(data):
    new_list = [' '.join([w for w in line.split() if not is_valid_date(w)]) for line in data]
    return (new_list[0])

def stemmer_and_stopWord(doc):
    
    doc= nlp(doc)
    token_list = []
    for token in doc:
        lemma = token.lemma_
        if lemma == '-PRON-' or lemma == 'be':
            lemma = token.text
        token_list.append(lemma)

    stemmed = token_list
    
    # Create list of word tokens after removing stopwords
    
    filtered_sentence =[] 
    for word in stemmed[:100]:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word)
    return (' '.join(filtered_sentence))

def normaliz(filtered_sentence):
        
    words = [str(word).lower() for word in filtered_sentence.split()]
    return  ' '.join(words[:100])

# For number removal ~

def numbers_removal(data):
    s = [data]
    result = ''.join([i for i in s if not i.isdigit()])
    return (result)
# For punchuation & double white spaces ~

def punch_removal(words):
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in [words]]
    return re.sub(' +', ' ', stripped[:100][0])


def cleaner(data):
    string = [data]
    string = date_removal(string)
    # string = numbers_removal(string)
    string = punch_removal(string)
    string = stemmer_and_stopWord(string)
    string = normaliz(string)
    return string

In [ ]:
def cleaner(data):
    string = [data]
    string = date_removal(string)
    string = numbers_removal(string)
    string = punch_removal(string)
    string = stemmer_and_stopWord(string)
    string = normaliz(string)
    return string

print('* --------------- for train data --------------- *')

list=[]
for i in train['excerpt']: # cleansing !!
    list.append(cleaner(i))

train['excerpt']=pd.Series(list) # updating the attributee !!

print('* --------------- for test data --------------- *')

list=[]
for i in test['excerpt']: # cleansing !!
    list.append(cleaner(i))

test['excerpt']=pd.Series(list) # updating the attributee !!

In [ ]:
# import langdetect

# print('* ------------------------- for train data-------------------------------- *')

# train['lang'] = train["excerpt"].apply(lambda x: langdetect.detect(x) if x.strip() != "" else "")

# fig, ax = plt.subplots()
# fig.suptitle('lang', fontsize=12)
# train['lang'].reset_index().groupby('lang').count().sort_values(by= "index").plot(kind="barh", legend=False, ax=ax).grid(axis='x')
# plt.show()

# print('* -------------------------- for test data ------------------------------- *')

# test['lang'] = test["excerpt"].apply(lambda x: langdetect.detect(x) if x.strip() != "" else "")

# fig, ax = plt.subplots()
# fig.suptitle('lang', fontsize=12)
# test['lang'].reset_index().groupby('lang').count().sort_values(by= "index").plot(kind="barh", legend=False, ax=ax).grid(axis='x')
# plt.show()

In [ ]:
train.columns

In [ ]:
extra_features = ['word_count', 'char_count', 'num_unique_words', 'num_stopwords',
       'num_punctuations', 'num_words_upper', 'num_words_title',
       'mean_word_len']

In [ ]:
## Training :----------->

In [ ]:
test.head(2)

In [ ]:
## if It helped !! do care to Upvote !! :)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences 
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam



import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, MaxPooling1D, GlobalMaxPooling1D, LSTM, Dropout, GRU, Activation, Embedding, Bidirectional,SpatialDropout1D, BatchNormalization, Conv1D, MaxPool1D

In [ ]:
MAX_NB_WORDS = 28523 + 1
# Max number of words in each description.
MAX_SEQUENCE_LENGTH = 100
# This is fixed.

EMBEDDING_DIM = 100

Job_type = {
    'Permanent':0,
    'Contract/Interim':1,
    'Contract/Temp':2,
    'Temporary/Seasonal':3,
    'Any':4,
    'Part-Time':6
    }

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(train['excerpt'].values)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, 
                      filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', 
                      lower=True, oov_token='OOV')

tokenizer.fit_on_texts(train['excerpt'].values)
X = tokenizer.texts_to_sequences(train['excerpt'].values)

MAX_NB_WORDS = len(tokenizer.word_index) + 1 
# Max number of words in each description.
MAX_SEQUENCE_LENGTH = 100
# This is fixed.
EMBEDDING_DIM = 100

X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH) # Training labels

In [ ]:
X = tokenizer.texts_to_sequences(train['excerpt'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Y = train['target']
print('Shape of label tensor:', Y.shape)

In [ ]:
extra_features_train = train[extra_features].to_numpy(dtype='int32')
X = np.concatenate((X,extra_features_train), axis=1)

In [ ]:
## modeling :

In [ ]:
## Random Forest : !!!

# from sklearn.ensemble import RandomForestRegressor
# from sklearn import metrics

# X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.10, random_state=0 )

# '''parameters = {
#     "learning_rate": [0.01, 0.05, 0.1, 0.2],
#     "max_depth":[3,5,8],
#     "max_features":["log2","sqrt"],
#     "criterion": ["friedman_mse"],
#     "subsample":[0.5, 0.8, 0.9, 1.0],
#     "n_estimators":[100,500,200]
#     }'''


# reg = RandomForestRegressor(max_depth=8,n_estimators=200,n_jobs=2)
# reg.fit(X_train, y_train)

# y_pred=reg.predict(X_test)
# print('MAE:-->',metrics.mean_absolute_error(y_test, y_pred))
# print('MSE:-->',metrics.mean_squared_error(y_test, y_pred))
# print('RMSE:-->',np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# print('R2:-->',metrics.r2_score(y_test, y_pred))

In [ ]:
# from sklearn.preprocessing import StandardScaler

# X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
# scaler= StandardScaler() # before reshaping

# # print(X_test[0])

# X_train = scaler.fit_transform(X_train)
# X_test = scaler.fit_transform(X_test)

# # print(X_test[0])

# print(X_train.shape)
# print(X_test.shape)

# X_train=X_train.reshape(X_train.shape[0], X_train.shape[1],1) # it needs to be a nummpy array
# X_test=X_test.reshape(X_test.shape[0], X_test.shape[1],1)

# print(X_train.shape)
# print(X_test.shape)

In [ ]:
## convolution !!

# epochs=10
# cat_model=Sequential()

# cat_model.add(Conv1D(filters=32, activation='relu', kernel_size=2, input_shape=(103,1)))
# cat_model.add(Conv1D(filters=64, activation='relu', kernel_size=2))
# cat_model.add(Dropout(0.2))
# # cat_model.add(Conv1D(filters=128, activation='relu', kernel_size=2))
# # cat_model.add(BatchNormalization())
# # cat_model.add(Dropout(0.2))
# cat_model.add(Flatten())
# cat_model.add(Dense(128, activation='relu'))
# cat_model.add(Dropout(0.2))

# cat_model.add(Dense(1,activation='linear'))

# # cat_model.summary()
# cat_model.compile( optimizer=Adam(lr=0.05), loss='mae', metrics=['mae'])

# history=cat_model.fit(X_train,y_train, epochs=epochs, validation_data=(X_test,y_test), verbose=2)

# accr = cat_model.evaluate(X_test,y_test)
# print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
cat_model = Sequential()
cat_model.add(Embedding(MAX_NB_WORDS, 300, input_length=X.shape[1]))
cat_model.add(Bidirectional(LSTM(32, return_sequences=True)))
cat_model.add(Bidirectional(LSTM(64)))

# cat_model.add(LSTM(64))
cat_model.add(Dropout(0.2))
cat_model.add(Dense(64, activation='relu'))
cat_model.add(Dropout(0.2))

cat_model.add(Dense(1, activation='linear'))
cat_model.compile(loss='mse', optimizer=Adam(lr=0.05), metrics=['mae'])

In [ ]:
epochs = 10
history = cat_model.fit(X_train, Y_train, epochs=epochs,validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
accr = cat_model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
## Pedictions

In [ ]:
test_tok = tokenizer.texts_to_sequences(test['excerpt'].values)
test_tok = pad_sequences(test_tok, maxlen=MAX_SEQUENCE_LENGTH)

extra_features_test = test[extra_features].to_numpy(dtype='int32')
test_tok = np.concatenate((test_tok, extra_features_test), axis=1)

# test_tok = scaler.fit_transform(test_tok)
# test_tok = test_tok.reshape(7, 103,1)

In [ ]:
# target_pred = []
# extra_features_test = test[['word_count', 'char_count', 'avg_word_length']].to_numpy(dtype='int32')

# for j,i in enumerate(test['excerpt']):
#     seq = tokenizer.texts_to_sequences([i]) # Using the same fitted tokenizer 
#     padded = pad_sequences(seq, maxlen = MAX_SEQUENCE_LENGTH) # which was trained on the train data
#     padded = np.concatenate((padded,[extra_features_test[j]]), axis=1)
# #     target_pred.append(reg.predict(padded)[0])
# #     padded = scaler.fit_transform(padded)
#     padded=padded.reshape(1, 103,1)
#     target_pred.append(cat_model.predict(padded)[0][0])
    
target_pred = cat_model.predict(test_tok)
target_pred = [item for sublist in target_pred for item in sublist]

In [ ]:
# test_tok

In [ ]:
submission['target'] = pd.Series(target_pred)

submission

In [ ]:
submission.to_csv("submission.csv",index=False)